## Convert LabelMe segmentations to YOLO format

In [9]:
import os
import json
import cv2
import shutil

In [10]:
# Example usage
input_dir=r"C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\GSV\Wagga\Panos_clipped_higher_res"  # Directory with JSON files
# input_dir=r"C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\RICS\RICS_wagga_sample_images"  # Directory with JSON files
output_dir = r"C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\GSV\Wagga\YOLO_test\finetune\all"  # Base output directory
# output_dir = r"C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\RICS\YOLO_test"  # Base output directory
# Define the fixed class list directly in the code
# original_class_list  = ["_background_", "foundation", "partial door", "front door", "partial garage door", "stairs","garage door"]
original_class_list  = ["foundation", "front door", "stairs","garage door"]

In [11]:
# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
images_dir = os.path.join(output_dir, "images")
labels_dir = os.path.join(output_dir, "labels")
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

In [12]:
# Process each JSON file
for file_name in os.listdir(input_dir):
    if file_name.endswith(".json"):
        json_path = os.path.join(input_dir, file_name)
        img_path=json_path.replace('.json','.jpg')

        image_output_path = os.path.join(images_dir, f"{file_name.replace('.json', '.jpg')}")
        yolo_label_file = os.path.join(labels_dir, file_name.replace('.json', '.txt'))
        if os.path.exists(image_output_path):
            print('file exists, skipping...')
        else:
            img = cv2.imread(img_path)
            height, width, _ = img.shape
            with open(json_path, 'r') as f:
                data = json.load(f)
            with open(yolo_label_file, 'w') as out_file:
                for shape in data['shapes']:
                    points = shape['points']
                    class_name = shape['label']
                    
                    if class_name not in original_class_list:
                        continue
                    
                    class_id = original_class_list.index(class_name)
                    
                    # Extract bounding box from segmentation points
                    x_min = min(p[0] for p in points)
                    y_min = min(p[1] for p in points)
                    x_max = max(p[0] for p in points)
                    y_max = max(p[1] for p in points)
                    
                    # Normalize to YOLO format
                    x_center = (x_min + x_max) / 2 / width
                    y_center = (y_min + y_max) / 2 / height
                    w_bbox = (x_max - x_min) / width
                    h_bbox = (y_max - y_min) / height
                    
                    # Write to YOLO format
                    out_file.write(f"{class_id} {x_center} {y_center} {w_bbox} {h_bbox}\n")

                # copy over image file
                shutil.copy(img_path, image_output_path)

print(f"Processing completed. Images saved to {images_dir}, labels saved to {labels_dir}.")

Processing completed. Images saved to C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\GSV\Wagga\YOLO_test\finetune\all\images, labels saved to C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\GSV\Wagga\YOLO_test\finetune\all\labels.
